# Lake model continued

In the previous week you used the lake problem as a means of getting aquinted with the workbench. In this assignment we will continue with the lake problem, focussing explicitly on using it for open exploration. You can use the second part of [this tutorial](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html) for help.

**It is paramount that you are using the lake problem with 100 decision variables, rather than the one found on the website with the seperate anthropogenic release decision**

## Apply sensitivity analysis
There is substantial support in the ema_workbench for global sensitivity. For this, the workbench relies on [SALib](https://salib.readthedocs.io/en/latest/) and feature scoring which is a machine learning alternative for global sensitivity analysis. 


1. Apply Sobol with 3 seperate release policies (0, 0.05, and 0.1) and analyse the results for each release policy seperately focusing on the reliability objective. Do the sensitivities change depending on the release policy? Can you explain why or why not?

*hint: you can use sobol sampling for the uncertainties, and set policies to a list with the 3 different release policies. Next, for the analysis using logical indexing on the experiment.policy column you can select the results for each seperate release policy and apply sobol to each of the three seperate release policies. If this sounds too complicated, just do it on each release policy seperately.*



2. Repeat the above analysis for the 3 release policies but now with extra trees feature scoring and for all outcomes of interest. As a bonus, use the sobol experiment results as input for extra trees, and compare the results with those resulting from latin hypercube sampling.

*hint: you can use [seaborn heatmaps](https://seaborn.pydata.org/generated/seaborn.heatmap.html) for a nice figure of the results*


In [1]:
import math
import matplotlib.pyplot as plt

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis import dimensional_stacking
import ema_workbench
import numpy as np
import pandas as pd
from scipy.optimize import brentq

from SALib.analyze import sobol

from ema_workbench import (Model, RealParameter, ScalarOutcome, Constant,
                           ema_logging, MultiprocessingEvaluator, Policy,
                           SequentialEvaluator)
from ema_workbench.em_framework.evaluators import SOBOL
from ema_workbench.em_framework import get_SALib_problem

from lakemodel_function import lake_problem
import seaborn as sns

ModuleNotFoundError: No module named 'ema_workbench'

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

# instantiate the model
lake_model = Model('lakeproblem', function=lake_problem)
lake_model.time_horizon = 100

# specify uncertainties
lake_model.uncertainties = [RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2.0, 4.5),
                            RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('delta', 0.93, 0.99)]

# set levers, one for each time step
lake_model.levers = [RealParameter(str(i), 0, 0.1) for i in
                     range(lake_model.time_horizon)]

# specify outcomes
lake_model.outcomes = [ScalarOutcome('max_P'),
                       ScalarOutcome('utility'),
                       ScalarOutcome('inertia'),
                       ScalarOutcome('reliability')]

# override some of the defaults of the model
lake_model.constants = [Constant('alpha', 0.41),
                        Constant('nsamples', 150)]

# generate sa single default no release policy
policies = [Policy('no release'  , **{'l'+str(i): 0.01 for i in range(100)}),
          Policy('some release', **{'l'+str(i): 0.05 for i in range(100)}),
          Policy('more release', **{'l'+str(i): 0.1  for i in range(100)})]

n_scenarios = 1000

# Use Multiprocess in pycharm, this will take a long time.
for policy in policies:
    with SequentialEvaluator(lake_model) as evaluator:
        x, y = evaluator.perform_experiments(n_scenarios, policies=policy,
                                             uncertainty_sampling=SOBOL)
    
    # Remove all the levers from the plot
    x = x.iloc[:, :8]
    fs = feature_scoring.get_feature_scores_all(x, y)
    sns.heatmap(fs, cmap='viridis', annot=True)
    plt.show()

The plots from the three different policies indicate that there are only very minor changes between the dependencies of the three policies, which could also be attributed to uncertainties in general. Overal, `max_P` becomes more dependent on `b` and `q` throughout the three policies, becoming less dependent on `stdev` and `delta`